In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json
import time

# Google API Key
from config import gkey

# GETTING DATA

### This is a csv of 1,673 clinics that are recognized or are in the process of being recognized as diabetes treatment centers. 

In [ ]:
# read csv
treatment = pd.read_csv("../data/DPRP_Results_Full_RegistryAug_05_2020.csv",encoding='cp1252')
treatment.head()

In [ ]:
# add lat and lng columns
treatment['Lat'] = ""
treatment['Lng'] = ""

# check number of rows
treatment.shape

# START OF API CALL

In [ ]:
# keep a counter
count = 0
set_num = 0

# for loop to iterate through rows
for index, row in treatment.iterrows():
    row = row.copy() 
    address = row['Address Line 1']
    
    # address doesn't work for suites, and prob po box, and some other types of addresses so expect to see it in errors.
    params = {"address": address,"key": gkey}
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    response = requests.get(base_url, params=params).json()
    
    # pull the lat and lng for df and print any that don't have 
    try:
        lat = response["results"][0]["geometry"]["location"]["lat"]
        lng = response["results"][0]["geometry"]["location"]["lng"]
        treatment.loc[index, 'Lat'] = lat
        treatment.loc[index, 'Lng'] = lng
    except IndexError:
        print(f"{index} - '{address}' not found. Skipping ... ")
        continue
    
    # add a time sleep
    if response == 45:
        t1 = time.time()
        count += 1
        set_num +=1
        time.sleep(60-(t1-t0))
        t0 = time.time()

# END OF API CALL

# SAVING NEW DF TO CSV

In [ ]:
# checking that all that went through
treatment.head()
treatment.tail()

#checking how many rows have post api call
treatment.shape

## DON'T RUN THIS IT'S ALREADY SAVED
### save new df to csv. remove index & keep the header
treatment.to_csv("output/treatment_coord.csv", index=False, header=True)

# CLEANING NEW CSV

In [2]:
# read new csv
treatment_coord = pd.read_csv("output/treatment_coord.csv")
treatment_coord.head()

,Name,Address Line 1,Address Line 2,City,State,Zip,Phone,Web Address,Recognition Status,Class open To,Delivery Mode,Lat,Lng
0,Anchorage Neighborhood Health Center,4951 Business Park Blvd.,NaN,Anchorage,AK,99503,(907) 743-7200,NaN,Pending,Public Employees Other,In-Person,61.175943,-149.887679
1,University of Alaska Fairbanks Cooperative Ext...,1675 C Street,NaN,Anchorage,AK,99501,(907) 786-6300,http://www.uaf.edu/ces/districts/anchorage/,Full,Public,In-Person,61.205926,-149.886380
2,YMCA of Anchorage,5353 Lake Otis Parkway,NaN,Anchorage,AK,99507,(907) 563-3211 Ext: 104,http://www.ymcaalaska.org,Full,Public,In-Person,61.172156,-149.834741
3,Tanana Chiefs Conference,Chief Andrew Isaac Health Center,1717 West Cowles St,Fairbanks,AK,99701,(907) 451-6682 Ext: 3768,NaN,Preliminary,Employees Members Other,In-Person,64.832476,-147.745224
4,Tanana Chiefs Conference,1717 W. Cowles St.,NaN,Fairbanks,AK,99701,(907) 451-6682 Ext: 3890,NaN,Preliminary,Other,Combination,64.832514,-147.744362


In [3]:
# rename columns. keep lat and lng in cap title letter for tableau
treatment_coord = treatment_coord.rename(columns={"Name":"name",
                                        "Address Line 1": "address_line1",
                                        "Address Line 2": "address_line2",
                                        "City":"city","State":"state", "Zip":"zip","Phone":"phone",
                                        "Web Address": "website",
                                        "Recognition Status": "recognition_status",
                                        "Class open To": "patient_type",
                                        "Delivery Mode": "delivery_mode","Lat":"Latitude","Lng":"Longitude"})
treatment_coord.head()

,name,address_line1,address_line2,city,state,zip,phone,website,recognition_status,patient_type,delivery_mode,Latitude,Longitude
0,Anchorage Neighborhood Health Center,4951 Business Park Blvd.,NaN,Anchorage,AK,99503,(907) 743-7200,NaN,Pending,Public Employees Other,In-Person,61.175943,-149.887679
1,University of Alaska Fairbanks Cooperative Ext...,1675 C Street,NaN,Anchorage,AK,99501,(907) 786-6300,http://www.uaf.edu/ces/districts/anchorage/,Full,Public,In-Person,61.205926,-149.886380
2,YMCA of Anchorage,5353 Lake Otis Parkway,NaN,Anchorage,AK,99507,(907) 563-3211 Ext: 104,http://www.ymcaalaska.org,Full,Public,In-Person,61.172156,-149.834741
3,Tanana Chiefs Conference,Chief Andrew Isaac Health Center,1717 West Cowles St,Fairbanks,AK,99701,(907) 451-6682 Ext: 3768,NaN,Preliminary,Employees Members Other,In-Person,64.832476,-147.745224
4,Tanana Chiefs Conference,1717 W. Cowles St.,NaN,Fairbanks,AK,99701,(907) 451-6682 Ext: 3890,NaN,Preliminary,Other,Combination,64.832514,-147.744362


In [4]:
# edit NaNs in address line 2 & website 
treatment_coord['address_line2'] = treatment_coord['address_line2'].fillna("None")
treatment_coord['website'] = treatment_coord['website'].fillna("Unavailable")
treatment_coord.head()

,name,address_line1,address_line2,city,state,zip,phone,website,recognition_status,patient_type,delivery_mode,Latitude,Longitude
0,Anchorage Neighborhood Health Center,4951 Business Park Blvd.,None,Anchorage,AK,99503,(907) 743-7200,Unavailable,Pending,Public Employees Other,In-Person,61.175943,-149.887679
1,University of Alaska Fairbanks Cooperative Ext...,1675 C Street,None,Anchorage,AK,99501,(907) 786-6300,http://www.uaf.edu/ces/districts/anchorage/,Full,Public,In-Person,61.205926,-149.886380
2,YMCA of Anchorage,5353 Lake Otis Parkway,None,Anchorage,AK,99507,(907) 563-3211 Ext: 104,http://www.ymcaalaska.org,Full,Public,In-Person,61.172156,-149.834741
3,Tanana Chiefs Conference,Chief Andrew Isaac Health Center,1717 West Cowles St,Fairbanks,AK,99701,(907) 451-6682 Ext: 3768,Unavailable,Preliminary,Employees Members Other,In-Person,64.832476,-147.745224
4,Tanana Chiefs Conference,1717 W. Cowles St.,None,Fairbanks,AK,99701,(907) 451-6682 Ext: 3890,Unavailable,Preliminary,Other,Combination,64.832514,-147.744362


In [5]:
# reorder columns to correspond with tableau's heirarchy
treatment_coord = treatment_coord[["name","address_line1","address_line2","city","state","zip","phone","Latitude","Longitude",
                                    "website","recognition_status","patient_type","delivery_mode"]]
treatment_coord.head()

,name,address_line1,address_line2,city,state,zip,phone,Latitude,Longitude,website,recognition_status,patient_type,delivery_mode
0,Anchorage Neighborhood Health Center,4951 Business Park Blvd.,None,Anchorage,AK,99503,(907) 743-7200,61.175943,-149.887679,Unavailable,Pending,Public Employees Other,In-Person
1,University of Alaska Fairbanks Cooperative Ext...,1675 C Street,None,Anchorage,AK,99501,(907) 786-6300,61.205926,-149.886380,http://www.uaf.edu/ces/districts/anchorage/,Full,Public,In-Person
2,YMCA of Anchorage,5353 Lake Otis Parkway,None,Anchorage,AK,99507,(907) 563-3211 Ext: 104,61.172156,-149.834741,http://www.ymcaalaska.org,Full,Public,In-Person
3,Tanana Chiefs Conference,Chief Andrew Isaac Health Center,1717 West Cowles St,Fairbanks,AK,99701,(907) 451-6682 Ext: 3768,64.832476,-147.745224,Unavailable,Preliminary,Employees Members Other,In-Person
4,Tanana Chiefs Conference,1717 W. Cowles St.,None,Fairbanks,AK,99701,(907) 451-6682 Ext: 3890,64.832514,-147.744362,Unavailable,Preliminary,Other,Combination


In [6]:
# looking to see how class is split up...want to try to divide it by adding new rows...
treatment_coord["patient_type"].value_counts()

Public                             947
Public Employees                   189
Public Employees Members           109
Employees                           93
Other                               78
Public Other                        56
Members                             51
Public Employees Other              44
Public Employees Members Other      41
Public Members                      19
Employees Other                     17
Employees Members                   13
Employees Members Other              6
Members Other                        6
Public Members Other                 4
Name: patient_type, dtype: int64

In [7]:
# splitting patient_type
treatment_split = pd.DataFrame(treatment_coord.patient_type.str.split(' ').tolist(), index=treatment_coord.name).stack()
# treatment_split

In [8]:
# reseting index in new df
treatment_split = treatment_split.reset_index([0, 'name'])
treatment_split

,name,0
0,Anchorage Neighborhood Health Center,Public
1,Anchorage Neighborhood Health Center,Employees
2,Anchorage Neighborhood Health Center,Other
3,Anchorage Neighborhood Health Center,
4,University of Alaska Fairbanks Cooperative Ext...,Public
...,...,...
4090,Powell Valley Health Care,Public
4091,Powell Valley Health Care,Employees
4092,Powell Valley Health Care,
4093,Goshen County Public Health,Public


In [9]:
# renameing columns in new df
treatment_split.columns = ['name', 'patient_type']
treatment_split

,name,patient_type
0,Anchorage Neighborhood Health Center,Public
1,Anchorage Neighborhood Health Center,Employees
2,Anchorage Neighborhood Health Center,Other
3,Anchorage Neighborhood Health Center,
4,University of Alaska Fairbanks Cooperative Ext...,Public
...,...,...
4090,Powell Valley Health Care,Public
4091,Powell Valley Health Care,Employees
4092,Powell Valley Health Care,
4093,Goshen County Public Health,Public


In [10]:
# merge treatment_split & treatment_coord
treatment_combo = pd.merge(treatment_split, treatment_coord, on="name",how="outer")
treatment_combo.head()

,name,patient_type_x,address_line1,address_line2,city,state,zip,phone,Latitude,Longitude,website,recognition_status,patient_type_y,delivery_mode
0,Anchorage Neighborhood Health Center,Public,4951 Business Park Blvd.,None,Anchorage,AK,99503,(907) 743-7200,61.175943,-149.887679,Unavailable,Pending,Public Employees Other,In-Person
1,Anchorage Neighborhood Health Center,Employees,4951 Business Park Blvd.,None,Anchorage,AK,99503,(907) 743-7200,61.175943,-149.887679,Unavailable,Pending,Public Employees Other,In-Person
2,Anchorage Neighborhood Health Center,Other,4951 Business Park Blvd.,None,Anchorage,AK,99503,(907) 743-7200,61.175943,-149.887679,Unavailable,Pending,Public Employees Other,In-Person
3,Anchorage Neighborhood Health Center,,4951 Business Park Blvd.,None,Anchorage,AK,99503,(907) 743-7200,61.175943,-149.887679,Unavailable,Pending,Public Employees Other,In-Person
4,University of Alaska Fairbanks Cooperative Ext...,Public,1675 C Street,None,Anchorage,AK,99501,(907) 786-6300,61.205926,-149.886380,http://www.uaf.edu/ces/districts/anchorage/,Full,Public,In-Person


In [12]:
# drop patient_type_y
treatment_combo = treatment_combo.drop(columns='patient_type_y')

# verify patient_type_y is dropped
treatment_combo.shape # output is:(4800, 13)
treatment_combo.head()

,name,patient_type_x,address_line1,address_line2,city,state,zip,phone,Latitude,Longitude,website,recognition_status,delivery_mode
0,Anchorage Neighborhood Health Center,Public,4951 Business Park Blvd.,None,Anchorage,AK,99503,(907) 743-7200,61.175943,-149.887679,Unavailable,Pending,In-Person
1,Anchorage Neighborhood Health Center,Employees,4951 Business Park Blvd.,None,Anchorage,AK,99503,(907) 743-7200,61.175943,-149.887679,Unavailable,Pending,In-Person
2,Anchorage Neighborhood Health Center,Other,4951 Business Park Blvd.,None,Anchorage,AK,99503,(907) 743-7200,61.175943,-149.887679,Unavailable,Pending,In-Person
3,Anchorage Neighborhood Health Center,,4951 Business Park Blvd.,None,Anchorage,AK,99503,(907) 743-7200,61.175943,-149.887679,Unavailable,Pending,In-Person
4,University of Alaska Fairbanks Cooperative Ext...,Public,1675 C Street,None,Anchorage,AK,99501,(907) 786-6300,61.205926,-149.886380,http://www.uaf.edu/ces/districts/anchorage/,Full,In-Person


In [13]:
# drop any nulls
drop_it = treatment_combo.dropna(how="any")
drop_it #output is:(4673, 14)

,name,patient_type_x,address_line1,address_line2,city,state,zip,phone,Latitude,Longitude,website,recognition_status,delivery_mode
0,Anchorage Neighborhood Health Center,Public,4951 Business Park Blvd.,None,Anchorage,AK,99503,(907) 743-7200,61.175943,-149.887679,Unavailable,Pending,In-Person
1,Anchorage Neighborhood Health Center,Employees,4951 Business Park Blvd.,None,Anchorage,AK,99503,(907) 743-7200,61.175943,-149.887679,Unavailable,Pending,In-Person
2,Anchorage Neighborhood Health Center,Other,4951 Business Park Blvd.,None,Anchorage,AK,99503,(907) 743-7200,61.175943,-149.887679,Unavailable,Pending,In-Person
3,Anchorage Neighborhood Health Center,,4951 Business Park Blvd.,None,Anchorage,AK,99503,(907) 743-7200,61.175943,-149.887679,Unavailable,Pending,In-Person
4,University of Alaska Fairbanks Cooperative Ext...,Public,1675 C Street,None,Anchorage,AK,99501,(907) 786-6300,61.205926,-149.886380,http://www.uaf.edu/ces/districts/anchorage/,Full,In-Person
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,Powell Valley Health Care,Public,777 Ave. H,None,Powell,WY,82435,(307) 754-2267 Ext: 3604,44.756996,-108.771452,Unavailable,Pending,In-Person
4796,Powell Valley Health Care,Employees,777 Ave. H,None,Powell,WY,82435,(307) 754-2267 Ext: 3604,44.756996,-108.771452,Unavailable,Pending,In-Person
4797,Powell Valley Health Care,,777 Ave. H,None,Powell,WY,82435,(307) 754-2267 Ext: 3604,44.756996,-108.771452,Unavailable,Pending,In-Person
4798,Goshen County Public Health,Public,2025 Campbell Dr. Suite #1,None,Torrington,WY,82240,(307) 532-4069,33.501088,-112.037390,Unavailable,Pending,In-Person


In [14]:
# rename patient_type_x
treatment1 = drop_it.rename(columns={"patient_type_x":"patient_type"})
treatment1

,name,patient_type,address_line1,address_line2,city,state,zip,phone,Latitude,Longitude,website,recognition_status,delivery_mode
0,Anchorage Neighborhood Health Center,Public,4951 Business Park Blvd.,None,Anchorage,AK,99503,(907) 743-7200,61.175943,-149.887679,Unavailable,Pending,In-Person
1,Anchorage Neighborhood Health Center,Employees,4951 Business Park Blvd.,None,Anchorage,AK,99503,(907) 743-7200,61.175943,-149.887679,Unavailable,Pending,In-Person
2,Anchorage Neighborhood Health Center,Other,4951 Business Park Blvd.,None,Anchorage,AK,99503,(907) 743-7200,61.175943,-149.887679,Unavailable,Pending,In-Person
3,Anchorage Neighborhood Health Center,,4951 Business Park Blvd.,None,Anchorage,AK,99503,(907) 743-7200,61.175943,-149.887679,Unavailable,Pending,In-Person
4,University of Alaska Fairbanks Cooperative Ext...,Public,1675 C Street,None,Anchorage,AK,99501,(907) 786-6300,61.205926,-149.886380,http://www.uaf.edu/ces/districts/anchorage/,Full,In-Person
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,Powell Valley Health Care,Public,777 Ave. H,None,Powell,WY,82435,(307) 754-2267 Ext: 3604,44.756996,-108.771452,Unavailable,Pending,In-Person
4796,Powell Valley Health Care,Employees,777 Ave. H,None,Powell,WY,82435,(307) 754-2267 Ext: 3604,44.756996,-108.771452,Unavailable,Pending,In-Person
4797,Powell Valley Health Care,,777 Ave. H,None,Powell,WY,82435,(307) 754-2267 Ext: 3604,44.756996,-108.771452,Unavailable,Pending,In-Person
4798,Goshen County Public Health,Public,2025 Campbell Dr. Suite #1,None,Torrington,WY,82240,(307) 532-4069,33.501088,-112.037390,Unavailable,Pending,In-Person


In [15]:
# reorganize df
treatment1 = treatment1[["name","address_line1","address_line2","city","state","zip","phone","Latitude","Longitude",
                                    "website","recognition_status","patient_type","delivery_mode"]]
treatment1.head()

,name,address_line1,address_line2,city,state,zip,phone,Latitude,Longitude,website,recognition_status,patient_type,delivery_mode
0,Anchorage Neighborhood Health Center,4951 Business Park Blvd.,None,Anchorage,AK,99503,(907) 743-7200,61.175943,-149.887679,Unavailable,Pending,Public,In-Person
1,Anchorage Neighborhood Health Center,4951 Business Park Blvd.,None,Anchorage,AK,99503,(907) 743-7200,61.175943,-149.887679,Unavailable,Pending,Employees,In-Person
2,Anchorage Neighborhood Health Center,4951 Business Park Blvd.,None,Anchorage,AK,99503,(907) 743-7200,61.175943,-149.887679,Unavailable,Pending,Other,In-Person
3,Anchorage Neighborhood Health Center,4951 Business Park Blvd.,None,Anchorage,AK,99503,(907) 743-7200,61.175943,-149.887679,Unavailable,Pending,,In-Person
4,University of Alaska Fairbanks Cooperative Ext...,1675 C Street,None,Anchorage,AK,99501,(907) 786-6300,61.205926,-149.886380,http://www.uaf.edu/ces/districts/anchorage/,Full,Public,In-Person


In [ ]:
# remove empty cells
